In [1]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("/mnt/d")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch

SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_PTH = join(join(DATA_HOME, "sale_model"), "sale_model.pth")


In [2]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt


I, H = 71, 33
epochs = 90

model = LSTM(71, 33, batch_first=True).cuda()
model.load_state_dict(torch.load(MODEL_PTH))
sd = Sales_Dataset(SALE_HOME, is_train=False)
dl = DataLoader(sd, batch_size=1)

In [13]:
import pandas as pd
cols = [ "store_nbr", "date","family"]
ans_df = pd.DataFrame()
f_dict = sorted(list(sd.family_set))
h, c = torch.randn(1,1, 33, dtype=torch.float32).cuda(), torch.randn(1,1, 33, dtype=torch.float32).cuda()

for X, y in dl:
    yhat, (_, _) = model(X, (h, c))
    yhat = yhat[0]
    start_date = pd.to_datetime("2017-08-16")
    for i in range(-16, 0):
        for j in range(yhat.shape[1]):
            f_dict = pd.concat((ans_df, pd.DataFrame({"store_nbr": [int(y.item())], "family": [f_dict[j]], "sale": [yhat[i][j].item()], "date": [start_date]})))
        start_date += pd.Timedelta(days=1)